<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/vegetation_anomaly_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Calculating Vegetation Index Anomalies for a Specified Region of Interest**


---


Original GEE Script Author: Aaron Eubank

Python Script Adaptation: Wynnie Gross


---


*   This script is run primarily to get a map of a specified vegetation index (NDVI, VCI, SAVI) anomalies compared to a baseline date range.
* This script defines a function to calculate this for a given region of interest.   
* The default dataset is MODIS 8-day product, however it is parameterizable to choose a different dataset.
*   Methodology adapted from GEARS - Geospatial Ecology and Remote Sensing - https://www.geospatialecology.com

In [8]:
import geemap
import ee
from google.colab import userdata
from datetime import datetime

ee.Authenticate()
ee.Initialize(project=userdata.get('projectname'))

def calculate_veg_anomaly(
    region_of_interest,
    start_date_baseline,
    end_date_baseline,
    start_date_recent,
    end_date_recent,
    veg_index='NDVI', # User specifies 'NDVI', 'EVI', or 'SAVI'
    source='MODIS',  # User specifies 'MODIS', 'Landsat', or 'Sentinel-2'
    output_type='percentage',
    mask_cropland=True,
    cloud_masking=True
):
    """
    Calculates Vegetation Index anomalies for a given region and time period,
    compared to a baseline period.

    Args:
        region_of_interest: ee.FeatureCollection. The area to analyze.
        start_date_baseline (str): Start date for the baseline period (YYYY-MM-DD).
        end_date_baseline (str): End date for the baseline period (YYYY-MM-DD).
        start_date_recent (str): Start date for the recent period (YYYY-MM-DD).
        end_date_recent (str): End date for the recent period (YYYY-MM-DD).
        veg_index (str): Vegetation index to use ('NDVI', 'EVI', or 'SAVI').
        source (str): Source dataset ('MODIS', 'Landsat', 'Sentinel-2').
        output_type (str): 'percentage' for percentage anomaly, 'raw' for raw difference.
        mask_cropland (bool): Whether to apply a cropland mask.
        cloud_masking (bool): Whether to apply cloud masking.

    Returns:
        tuple: (ee.Image, geemap.Map) The mean vegetation index anomaly image and the map object.
    """

    # Automatically set dataset and bands based on source input
    if source == 'MODIS':
        dataset_id = 'MODIS/061/MOD09GA'
        nir_band = 'sur_refl_b02'
        red_band = 'sur_refl_b01'
        blue_band = 'sur_refl_b03'
    elif source == 'Sentinel-2':
        dataset_id = 'COPERNICUS/S2_SR'
        nir_band = 'B8'
        red_band = 'B4'
        blue_band = 'B2'
    elif source == 'Landsat':
        dataset_id = 'LANDSAT/LC08/C02/T1_L2'
        nir_band = 'SR_B5'
        red_band = 'SR_B4'
        blue_band = 'SR_B2'
    else:
        raise ValueError("Unsupported source. Choose from 'MODIS', 'Landsat', or 'Sentinel-2'.")

    # Load the image collection
    collection = ee.ImageCollection(dataset_id).filterBounds(region_of_interest)

    # Cloud Mask function
    def mask_clouds(image):
        if source == 'MODIS':
            QA = image.select(['state_1km'])
            bitMask = 1 << 10
            return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))
        elif source == 'Sentinel-2':
            return image.updateMask(image.select('QA60').eq(0))
        elif source == 'Landsat':
            qa = image.select('QA_PIXEL')
            cloudMask = qa.bitwiseAnd(1 << 3).eq(0)
            return image.updateMask(cloudMask)
        else:
            return image  # No cloud masking for other datasets

    # Crop Mask
    def crop_mask(image):
        esa = ee.ImageCollection('ESA/WorldCover/v100')
        esa_latest = esa.limit(1, 'system:time_start').first()
        cropland = esa_latest.updateMask(
            esa_latest.select('Map').eq(40).clip(region_of_interest)
        )
        return image.updateMask(cropland)

    # Function to add vegetation index as a band
    def add_veg_index(image):
        if veg_index == 'NDVI':
            index = image.normalizedDifference([nir_band, red_band])
        elif veg_index == 'EVI':
            index = image.expression(
                "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
                {
                    "NIR": image.select(nir_band),
                    "RED": image.select(red_band),
                    "BLUE": image.select(blue_band),
                },
            )
        elif veg_index == 'SAVI':
            L = 0.5
            index = image.expression(
                "((NIR - RED) * (1 + L)) / (NIR + RED + L)",
                {
                    "NIR": image.select(nir_band),
                    "RED": image.select(red_band),
                    "L": L,
                },
            )
        else:
            raise ValueError("Unsupported vegetation index")

        return image.addBands(index.rename(veg_index.lower()))

    # Get data for baseline date range
    baseline_collection = collection.filterDate(
        start_date_baseline, end_date_baseline
    ).filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))

    if cloud_masking:
        baseline_collection = baseline_collection.map(mask_clouds)
    baseline_collection = baseline_collection.map(add_veg_index).select(veg_index.lower())

    # Check if baseline_collection is empty and print size
    print('Baseline Collection Size:', baseline_collection.size().getInfo())
    if baseline_collection.size().getInfo() == 0:
        raise ValueError("Baseline collection is empty. Check date range and filters.")

    baseline_mean = baseline_collection.mean().clip(region_of_interest)

    if mask_cropland:
        baseline_mean = baseline_mean.updateMask(crop_mask(baseline_mean))

    # Get data for recent date range
    recent_collection = collection.filterDate(
        start_date_recent, end_date_recent
    ).filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))

    if cloud_masking:
        recent_collection = recent_collection.map(mask_clouds)
    recent_collection = recent_collection.map(add_veg_index).select(veg_index.lower())

    # Check if recent_collection is empty and print size
    print('Recent Collection Size:', recent_collection.size().getInfo())
    if recent_collection.size().getInfo() == 0:
        raise ValueError("Recent collection is empty. Check date range and filters.")

    if mask_cropland:
        recent_collection = recent_collection.map(crop_mask)

    # Subtract the baseline mean from the recent image
    def subtract_mean(image):
        return image.subtract(baseline_mean).copyProperties(
            image, ['system:time_start'])

    # Convert to a percentage anomaly
    def to_percentage(image):
        return image.divide(baseline_mean).multiply(100).copyProperties(
            image, ['system:time_start'])

    anomalies = recent_collection.map(subtract_mean)

    if output_type == 'percentage':
        anomalies = anomalies.map(to_percentage)

    # Find average anomaly
    mean_anomaly = anomalies.mean()

    # Create a map object and automatically center on centroid of ROI
    Map = geemap.Map(center=region_of_interest.geometry().centroid().getInfo()['coordinates'][::-1], zoom=7)

    # Visualization parameters
    veganoms_vis = {
        'min': -50,
        'max': 50,
        'palette': ['darkred', 'red', 'yellow', 'green', 'darkgreen']
    }

    # Add the anomaly layer to the map
    Map.addLayer(mean_anomaly, veganoms_vis, f'Mean {veg_index} Anomaly ({output_type})')

    # Adding Outline to the Map
    outline = region_of_interest.style(fillColor='00000000')
    Map.addLayer(outline, {}, "Region of Interest")

    # Add legend
    Map.add_colorbar(
        veganoms_vis, label=f"{veg_index} Anomaly {output_type.capitalize()}",
        layer_name="Mean Anomaly", orientation="horizontal"
    )

    return mean_anomaly, Map

# Function to define region of interest based on country, admin1, and/or admin2

def get_region_of_interest(country, admin1_list=None, admin2_list=None):
    """
    Filters the admin2 boundaries dataset by country, and optionally by multiple admin1 and/or admin2 regions.

    Args:
        country (list): The name of the country (field 'admin0').
        admin1_list (list, optional): List of admin1 regions (field 'admin1').
        admin2_list (list, optional): List of admin2 regions (field 'admin2').

    Returns:
        ee.FeatureCollection: Filtered region of interest.
    """
    # Load the world admin2 boundaries dataset
    admin2_dataset = ee.FeatureCollection("projects/ee-aeubank/assets/world_admin2")

    # Start with a filter for the country
    filters = [ee.Filter.eq('admin0', country)]

    # Add filters for multiple admin1 regions if provided
    if admin1_list:
        admin1_filters = [ee.Filter.eq('admin1', region) for region in admin1_list]
        filters.append(ee.Filter.Or(*admin1_filters))

    # Add filters for multiple admin2 regions if provided
    if admin2_list:
        admin2_filters = [ee.Filter.eq('admin2', region) for region in admin2_list]
        filters.append(ee.Filter.Or(*admin2_filters))

    # Combine all filters using an AND filter
    combined_filter = ee.Filter.And(*filters)

    # Apply the combined filter to the dataset
    region_of_interest = admin2_dataset.filter(combined_filter)

    return region_of_interest

## EXAMPLE USAGE ##
region_of_interest = get_region_of_interest('Sudan', ['Sennar','Aj Jazirah'])
baseline_start = '2013-01-01'
baseline_end = '2022-12-31'
recent_start = '2023-01-01'
recent_end = datetime.now().isoformat()

# Calculate Anomaly and get map
anomaly_result, anomaly_map = calculate_veg_anomaly(
    region_of_interest,
    baseline_start,
    baseline_end,
    recent_start,
    recent_end,
    veg_index='NDVI',
    source='MODIS',
)

# Display the map
anomaly_map

Baseline Collection Size: 1818
Recent Collection Size: 366


Map(center=[12.89587933499483, 34.054257422974885], controls=(WidgetControl(options=['position', 'transparent_…